In [1]:
import math
import torch

from transformers import BertForMaskedLM, BertTokenizer, GPT2Tokenizer, GPT2LMHeadModel, RobertaForMaskedLM, RobertaTokenizer, \
XLMTokenizer, XLMWithLMHeadModel, ElectraTokenizer, ElectraForMaskedLM

gpu_id=0

tokenizer = GPT2Tokenizer.from_pretrained('gpt2-xl')
model = GPT2LMHeadModel.from_pretrained('gpt2-xl').to('cuda:'+str(gpu_id))
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1600)
    (wpe): Embedding(1024, 1600)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((1600,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1600,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((1600,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2):

In [228]:
       
#from pytorch_pretrained_bert import OpenAIGPTTokenizer, OpenAIGPTModel, OpenAIGPTLMHeadModel
# Load pre-trained model (weights)
#model = OpenAIGPTLMHeadModel.from_pretrained('openai-gpt')
#model.eval()
# Load pre-trained model tokenizer (vocabulary)
import time

def score_1(sentence):
    tokenize_input = tokenizer.encode(sentence)
    tensor_input = torch.tensor(tokenize_input,device='cuda:0')
    with torch.no_grad():
        loss, logits=model(tensor_input, labels=tensor_input,use_cache=False)
    n=len(tensor_input)
    return -loss.item()*(n-1)

def score_2(sentence):
    tokenize_input = tokenizer.encode(sentence)
    labels = torch.tensor(tokenize_input,device='cuda:0')
    with torch.no_grad():
        logits=model(labels,use_cache=False)[0]

    shift_logits = logits[..., :-1, :].contiguous()
    shift_labels = labels[..., 1:].contiguous()
    
    # Flatten the tokens
    loss_fct = torch.nn.CrossEntropyLoss(reduction='sum')
    loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))

    return -loss.item()


def score_3(sentence):
    tokenize_input = tokenizer.encode(sentence)
    labels = torch.tensor(tokenize_input,device='cuda:0')
    with torch.no_grad():
        logits=model(labels,use_cache=False)[0]
    
    shift_logits = logits[..., :-1, :].contiguous()
    shift_labels = labels[..., 1:].contiguous()
    
    log_softmax=torch.nn.LogSoftmax(dim=-1)(shift_logits)
    n=len(log_softmax)        
    log_softmax_inputs=torch.gather(log_softmax, -1, shift_labels.reshape([n,-1]))
            
    return log_softmax_inputs.sum().item()

torch.pi = torch.acos(torch.zeros(1)).item() * 2 # which is 3.1415927410125732

def typicality(sentence):
    tokenize_input = tokenizer.encode(sentence)
    labels = torch.tensor(tokenize_input,device='cuda:0')
    with torch.no_grad():
        logits=model(labels,use_cache=False)[0]
    
    shift_logits = logits[..., :-1, :].contiguous()
    shift_labels = labels[..., 1:].contiguous()
    
    log_softmax=torch.nn.LogSoftmax(dim=-1)(shift_logits)
    mean_log_p=log_softmax.mean(dim=-1,keepdim=True)
    
    n=len(log_softmax)        
    log_softmax_inputs=torch.gather(log_softmax, -1, shift_labels.reshape([n,-1]))
            
    t=torch.abs(log_softmax_inputs-mean_log_p)
            
    return -t.sum().item()




def typicality_2(sentence):
    tokenize_input = tokenizer.encode(sentence)
    labels = torch.tensor(tokenize_input,device='cuda:0')
    with torch.no_grad():
        logits=model(labels,use_cache=False)[0]
    
    shift_logits = logits[..., :-1, :].contiguous()
    shift_labels = labels[..., 1:].contiguous()
    
    log_softmax=torch.nn.LogSoftmax(dim=-1)(shift_logits)
    mean_log_p=log_softmax.mean(dim=-1,keepdim=True)
    std_log_p=log_softmax.std(dim=-1,keepdim=True)
    
    n=len(log_softmax)        
    log_softmax_inputs=torch.gather(log_softmax, -1, shift_labels.reshape([n,-1]))
            
    z=(log_softmax_inputs-mean_log_p)/std_log_p
    t=-torch.log(std_log_p)-0.5*math.log(2*math.pi)-0.5*(z**2)
                
    return t.sum().item()


a=['. Min maxed ich tap ist imp och das.',   
   '. Airplay ish ist imp och inter est ich.',
   '. Pre och est ist nicht ing var ich.',
   '. Min byproduct imp imp imp imp imp och.',
   '. Inch sores enlarge rapidly withstanding var och ich.',
   '. Busters ist nicht sic ist nicht sic ist.',
   '. Kiel Kiel Kiel Kiel Kiel Kiel est lieu.',
   '. Rescuers swiping pajamas Isis is shillings a series.',
   '. Rescuers scrambled for their selected transiting Bowers farted.',   
   '. Fearing sprain to clog preface this odd shillings.',
   '. Rescuers were launched relive belongings midterms and it.',
   '. Thus continuing to Social Distance and Stay in.',
   '. Before I showed myself as an obvious commie.',
   '. There are assisted suicide options in certain countries.',
  ]

# debug log-likelihood
#t=time.time(); print([score_1(i) for i in a]); print('t=',(time.time()-t)/len(a)*1000)
#t=time.time(); print([score_2(i) for i in a]); print('t=',(time.time()-t)/len(a)*1000)
#t=time.time(); print([score_3(i) for i in a]); print('t=',(time.time()-t)/len(a)*1000)

for s in a:
    log_prob=score_3(s)
    t1=typicality(s)
    t2=typicality_2(s)
    print('log-prob: {:6.1f} | typicality {:6.1f} | typicality_2 {:6.1f} | {}'.format(log_prob,t1,t2,s))


log-prob:  -77.5 | typicality -136.6 | typicality_2 -160.3 | . Min maxed ich tap ist imp och das.
log-prob:  -88.9 | typicality -125.8 | typicality_2 -142.7 | . Airplay ish ist imp och inter est ich.
log-prob:  -73.4 | typicality -132.9 | typicality_2 -164.5 | . Pre och est ist nicht ing var ich.
log-prob:  -70.6 | typicality  -91.6 | typicality_2 -116.4 | . Min byproduct imp imp imp imp imp och.
log-prob:  -90.5 | typicality -137.2 | typicality_2 -149.2 | . Inch sores enlarge rapidly withstanding var och ich.
log-prob:  -64.3 | typicality -223.8 | typicality_2 -328.9 | . Busters ist nicht sic ist nicht sic ist.
log-prob:  -52.4 | typicality -184.5 | typicality_2 -257.5 | . Kiel Kiel Kiel Kiel Kiel Kiel est lieu.
log-prob: -150.6 | typicality -159.2 | typicality_2 -184.0 | . Rescuers swiping pajamas Isis is shillings a series.
log-prob: -147.3 | typicality -121.3 | typicality_2 -135.1 | . Rescuers scrambled for their selected transiting Bowers farted.
log-prob: -128.4 | typicality -124

In [125]:
import inspect

print(inspect.getsource(model.forward))

    @add_start_docstrings_to_callable(GPT2_INPUTS_DOCSTRING)
    def forward(
        self,
        input_ids=None,
        past=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        use_cache=True,
    ):
        r"""
        labels (:obj:`torch.LongTensor` of shape :obj:`(batch_size, sequence_length)`, `optional`, defaults to :obj:`None`):
            Labels for language modeling.
            Note that the labels **are shifted** inside the model, i.e. you can set ``lm_labels = input_ids``
            Indices are selected in ``[-100, 0, ..., config.vocab_size]``
            All labels set to ``-100`` are ignored (masked), the loss is only
            computed for labels in ``[0, ..., config.vocab_size]``

    Return:
        :obj:`tuple(torch.FloatTensor)` comprising various elements depending on the configuration (:class:`~transformers.GPT2Config`) and inputs:
       

In [191]:
with open('sents10k.txt') as f:
    natural_sentences=['. '+ l.strip() for l in f]

for s in natural_sentences:
    log_prob=score_3(s)
    t=typicality(s)
    print('{} log-prob: {:.2f} typicality {:.2f}'.format(s,log_prob,t))

. Before I showed myself as an obvious commie. log-prob: -51.18 typicality -108.88
. It seems like that studio just gets it. log-prob: -38.26 typicality -108.48
. I would not work that into an application. log-prob: -39.89 typicality -110.58
. Cut off the breaks for a clean cut. log-prob: -41.41 typicality -96.38
. Take the opportunity whenever and wherever you can. log-prob: -28.28 typicality -127.41
. Came here for the obvious American English equivalent. log-prob: -53.25 typicality -88.05
. Gonna need a lot more detail than that. log-prob: -33.48 typicality -127.76
. I literally said this in my last post. log-prob: -31.70 typicality -115.02
. Thus continuing to Social Distance and Stay in. log-prob: -64.27 typicality -73.69
. Standards of quality are enforced on all posts. log-prob: -41.52 typicality -102.51
. It was added recently to counter fence cheese. log-prob: -56.23 typicality -87.25
. I definitely have a lot to think about. log-prob: -25.54 typicality -133.81
. Believe all w

KeyboardInterrupt: 